# Save and restore models

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

### Get an example dataset

We'll use the Fashion-MNIST dataset again to train our model to demonstrate saving weights. To speed up these demonstration runs, only use the first 1000 examples:

In [1]:
from __future__ import absolute_import, division, print_function

import os
import pathlib

import tensorflow as tf
from tensorflow import keras


keras.__version__


'2.3.0-tf'

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Let's build a simple model we'll use to demonstrate saving and loading weights.

In [3]:
# Returns a short sequential model
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
  
    model.compile(optimizer='Adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  
    return model


# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

The primary use case is to automatically save checkpoints *during* and at *the end* of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

`keras.callbacks.ModelCheckpoint` is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.

### Checkpoint callback usage

Train the model and pass it the `ModelCheckpoint` callback:

In [4]:
checkpoint_path = os.path.join("training_1", "cp.hdf5")
checkpoint_dir = os.path.dirname(checkpoint_path)
os.makedirs(checkpoint_dir, exist_ok=True)
# Create checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                              save_weights_only=True,
                                              verbose=1, save_freq="epoch")

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 704/1000 [====================>.........] - ETA: 0s - loss: 1.2991 - accuracy: 0.5540 
Epoch 00001: saving model to training_1/cp.hdf5
1000/1000 [==============================] - 1s 720us/sample - loss: 1.1594 - accuracy: 0.5980 - val_loss: 0.7832 - val_accuracy: 0.7090
Epoch 2/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.7890 - accuracy: 0.7230
Epoch 00002: saving model to training_1/cp.hdf5
1000/1000 [==============================] - 0s 167us/sample - loss: 0.7553 - accuracy: 0.7250 - val_loss: 0.7120 - val_accuracy: 0.7360
Epoch 3/10
 736/1000 [=====================>........] - ETA: 0s - loss: 0.5889 - accuracy: 0.7935
Epoch 00003: saving model to training_1/cp.hdf5
1000/1000 [==============================] - 0s 166us/sample - loss: 0.5793 - accuracy: 0.8000 - val_loss: 0.6181 - val_accuracy: 0.7830
Epoch 4/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.4991 - accuracy: 0.8224
Epoc

This creates a single collection of Keras checkpoint files that are updated at the end of each epoch:

In [5]:
!ls {checkpoint_dir}

cp.hdf5


Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 123us/sample - loss: 2.4428 - accuracy: 0.0890
Untrained model, accuracy:  8.90%


Then load the weights from the checkpoint, and re-evaluate:

In [7]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 54us/sample - loss: 0.6158 - accuracy: 0.7880
Restored model, accuracy: 78.80%


### Checkpoint callback options

The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:


In [8]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = os.path.join("training_2", "cp-{epoch:04d}.hdf5")
checkpoint_dir = os.path.dirname(checkpoint_path)
os.makedirs(checkpoint_dir, exist_ok=True)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.hdf5

Epoch 00010: saving model to training_2/cp-0010.hdf5

Epoch 00015: saving model to training_2/cp-0015.hdf5

Epoch 00020: saving model to training_2/cp-0020.hdf5

Epoch 00025: saving model to training_2/cp-0025.hdf5

Epoch 00030: saving model to training_2/cp-0030.hdf5

Epoch 00035: saving model to training_2/cp-0035.hdf5

Epoch 00040: saving model to training_2/cp-0040.hdf5

Epoch 00045: saving model to training_2/cp-0045.hdf5

Epoch 00050: saving model to training_2/cp-0050.hdf5


Now, have a look at the resulting checkpoints (sorting by modification date):

In [9]:
checkpoint_dir

'training_2'

In [10]:
# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.hdf5")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints

[PosixPath('training_2/cp-0005'),
 PosixPath('training_2/cp-0010'),
 PosixPath('training_2/cp-0015'),
 PosixPath('training_2/cp-0020'),
 PosixPath('training_2/cp-0025'),
 PosixPath('training_2/cp-0030'),
 PosixPath('training_2/cp-0035'),
 PosixPath('training_2/cp-0040'),
 PosixPath('training_2/cp-0045'),
 PosixPath('training_2/cp-0050')]

Note: the default Keras format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [11]:
model = create_model()
model.load_weights(latest+".hdf5")
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 126us/sample - loss: 0.7937 - accuracy: 0.8070
Restored model, accuracy: 80.70%


## What are these files?

The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

* One or more shards that contain your model's weights. 
* An index file that indicates which weights are stored in which shard.  

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the `Model.save_weights` method.

In [13]:
# Save the weights
os.makedirs("checkpoints", exist_ok=True)
model.save_weights(os.path.join("checkpoints", "my_checkpoint"))

# Restore the weights
model = create_model()
model.load_weights(os.path.join("checkpoints", "my_checkpoint"))

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 124us/sample - loss: 0.7937 - accuracy: 0.8070
Restored model, accuracy: 80.70%


## Save the entire model

The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model in Keras is very useful.

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob.

In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 330us/sample - loss: 1.1823 - accuracy: 0.5910
Epoch 2/5
1000/1000 [==============================] - 0s 78us/sample - loss: 0.6665 - accuracy: 0.7690
Epoch 3/5
1000/1000 [==============================] - 0s 78us/sample - loss: 0.5901 - accuracy: 0.8100
Epoch 4/5
1000/1000 [==============================] - 0s 77us/sample - loss: 0.4894 - accuracy: 0.8430
Epoch 5/5
1000/1000 [==============================] - 0s 80us/sample - loss: 0.4471 - accuracy: 0.8520


Now recreate the model from that file:

In [15]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [16]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 126us/sample - loss: 0.7135 - accuracy: 0.7430
Restored model, accuracy: 74.30%


This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration

### PLEASE RUN THIS COMMAND IF YOU FINISHED THE NOTEBOOK

In [ ]:
import os
temp=os.getpid()
!kill -9 $temp